<a href="https://colab.research.google.com/github/yachtsman/Coursera_Capstone/blob/master/Segmenting_and_Clustering_Neighborhoods_in_Toronto_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Segmenting and Clustering Neighborhoods in Toronto

# Mining and structuring the data

1.   Creating a new notebook
2.   Scraping data from Wikipedia
3.   Creating the dataframe
4.   Submiting the link on Github





1. Creating a new notebook

Importing basic dependencies

In [1]:
import pandas as pd
import numpy as np
print('basic dependencies imported')

basic dependencies imported


2. Scraping data from wikipedia

In [2]:
from bs4 import BeautifulSoup
import requests

print('BeautifulSoup is imported.')

BeautifulSoup is imported.


In [3]:
wiki_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(wiki_url,'lxml')
print('soup cooked')

soup cooked


In [4]:
wiki_table = soup.find('table',{'class':'wikitable sortable'})
wiki_rows = wiki_table.find_all('tr')
#wiki_rows

3. Creating the dataframe

In [5]:
data = []
for row in wiki_rows:
    data.append([t.text.strip()
    for t in row.find_all('td')]
    )
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df.drop([0], inplace=True) # to skip first rows 'none' 

print(df.shape)
df.head()

(180, 3)


,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Cleaning  the 'Not assigned' rows

In [6]:
df1 = df[df['Borough'] != 'Not assigned'].reset_index(drop=True) # to filter out 'Not assigned' rows

print(df1.shape)
df1.head()

(103, 3)


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


combinig the Neighborhoods with the equal postal codes 

In [7]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))
print(df2.shape)
df2.head()

(103, 2)


,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [8]:
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']
df3 = df2.reset_index()

**The final cleaned table looks like:**

In [10]:
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
print(df3.shape)
df3.head()

(103, 3)


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
